# Logistic Regression for Image Classification
# 
### This notebook demonstrates training a logistic regression model for classification tasks.

### 1. Import Libraries

In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

### 2. Load Data From CSV Files

In [11]:
def load_data(train_path, test_path):
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    X_train = train_df.drop(columns=["label"]).values
    y_train = train_df["label"].values

    X_test = test_df.drop(columns=["label"]).values
    y_test = test_df["label"].values

    return X_train, y_train, X_test, y_test

In [ ]:
train_path = "../Dataset/train.csv"
test_path = "../Dataset/test.csv"

print("Loading dataset...")
X_train, y_train, X_test, y_test = load_data(train_path, test_path)

print(f"Training samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")
print(f"Number of features: {X_train.shape[1]}")

### 3. Preprocess Data Using StandardScalar to Standardize Data

In [13]:
def preprocess(X_train, X_test):
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    return X_train, X_test, sc

In [ ]:
print("Preprocessing...")
X_train, X_test, sc = preprocess(X_train, X_test)
print("Preprocessing complete!")

### 4. Train Logistic Regression Model

In [15]:
def train_logistic_regression(X_train, y_train):
    classifier = LogisticRegression(max_iter=500, solver='lbfgs')
    classifier.fit(X_train, y_train)

    return classifier

In [ ]:
print("Training Logistic Regression...")
model = train_logistic_regression(X_train, y_train)
print("Training complete!")

### 5. Evaluate Model Performance

In [17]:
def evaluate(model, X_test, y_test):
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    cm = confusion_matrix(y_test, preds)

    return acc, cm

In [ ]:
print("Evaluating...")
acc, cm = evaluate(model, X_test, y_test)

print(f"\nAccuracy: {acc:.4f}")
print("\nConfusion Matrix:")
print(cm)